In [ ]:
import math
import torch

import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import pickle
import os
import mlflow
import mlflow.pytorch
import re
import logging
import spacy

from spacy.lang.en import English

In [ ]:
logging.basicConfig(filename='POSTransformer.log',level=logging.WARNING)

In [ ]:
datasets = os.listdir(".data/wikitext-2/wikitext-2/")

if "wiki.train.tokens" in datasets:
    print("Removing useless symbols from datasets...")
    pattern = r"[^\w'\.,;:\!\?]"
    for x in datasets:
        with open(f".data/wikitext-2/wikitext-2/{x}", "r") as _f:
            _f = _f.read()

        _f = re.sub(pattern, ' ', _f)

        with open(f".data/wikitext-2/wikitext-2/{x}", "w") as _w:
            _w = _w.write(_f)
    print("Cleaning finished")

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
params = {"lr":1.8,"epochs":100,"embed_dim":240,"heads":8,"layers":2,"seq_len":32,"batch_size":64, "name":"picci"}

In [ ]:
def generate_square_subsequent_mask(sz):
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        mask = mask.to("cuda")
        return mask

class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        x = self.dropout(x)
        x = x.to("cuda")
        return x

In [ ]:
class TransformerModel(nn.Module):
    """
    
    """
    def __init__(self, d_model, nhead=2, num_encoder_layers=1, ntokens=5000, output=50):
        super(TransformerModel, self).__init__()
        
        self.output = output
        self.num_encoder_layers = num_encoder_layers
        self.nhead = nhead
        self.d_model = d_model
        self.ntokens = ntokens
        self.emb = nn.Embedding(self.ntokens, self.d_model)
        self.posenc = PositionalEncoding(d_model, 0.5)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=1024)
        self.transformer = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.linear = nn.Linear(self.d_model, self.output)
        self.trans_activation = None
        
    def save_activations(self, epoch=0, batch=0):
        """
        epoch: The running epoch, this information will be added to the files' names.
        batch: The running batch, this information will be added to the files' names
        
        """
        try:
            
            for x in range(len(self.transformer.layers)):
                # save the activations of each transformer encoder layer.
                with open(f"/path/to/harddrive/{params['name']}-attention-{x}-{epoch}-{batch}.p", "wb") as _f:
                    pickle.dump(self.transformer.layers[x].attn_activation.cpu().detach().numpy(), _f)
                           
        except:
            logging.warning(f"Model: {params["name"]} Failed to save activations for batch {batch} in epoch {epoch}! No More space left on disk!")
        return 0
        
    
    def forward(self, x, tgt, epoch=0, batch=0):
        x = self.emb(x) #* math.sqrt(self.d_model)
        tgt = self.emb(tgt)
        x = self.posenc(x)
        x = self.transformer(x, generate_square_subsequent_mask(x.shape[0]))
        self.trans_activation = x
        if not self.training:
            self.save_activations(epoch=epoch, batch=batch)
        return self.linear(x)


In [ ]:
import torchtext
from torchtext.data.utils import get_tokenizer

TEXT = torchtext.data.Field(tokenize = "spacy", tokenizer_language="en", 
                            init_token=None, 
                            eos_token=None, 
                            stop_words = ["=", "==", "@", "@@", "<", ">", "@-@", "@,@","@.@","@_@", "<eos>", "<sos>", "<pad>"], 
                            lower=True)

train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)
TEXT.build_vocab(train_txt)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def batchify(data, bsz):
    data = TEXT.numericalize([data.examples[0].text])
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data

batch_size = 64
eval_batch_size = 128
train_data = batchify(train_txt, batch_size)
val_data = batchify(val_txt, eval_batch_size)
test_data = batchify(test_txt, eval_batch_size)

print(train_data.shape)

In [ ]:
"""
tags = [0]*len(train_txt.examples[0].text)

from tqdm import tqdm

for x in tqdm(range(len(train_txt.examples[0].text))):
    tags[x] = nlp(train_txt.examples[0].text[x])
"""

In [ ]:
"""
finaltags = [0]*len(train_txt.examples[0].text)
print(len(tags), len(finaltags))

for x in tqdm(range(len(tags))):
    for y in tags[x]:
        finaltags[x] = y.tag_

print(finaltags[:50])
"""

In [ ]:
"""
print(len(finaltags))
import pickle
with open('wikitext.train2.pickle', 'wb') as handle:
    pickle.dump(finaltags, handle, protocol=pickle.HIGHEST_PROTOCOL)
"""

In [ ]:
with open("wikitext.train2.pickle", "rb") as h:
    traintags = pickle.load(h)
    
with open("wikitext.valid.pickle", "rb") as h:
    valtags = pickle.load(h)

In [ ]:
counter = 0
itotags = {}
tagstoi = {}
for x in nlp.get_pipe("tagger").labels:
    itotags[counter] = x
    tagstoi[x] = counter
    counter += 1
ntags = len(tagstoi.keys())

In [ ]:
traintags = torch.Tensor([tagstoi[x] for x in traintags])
valtags = torch.Tensor([tagstoi[x] for x in valtags])
print(traintags.shape, valtags.shape)
train_data.shape

In [ ]:
print(traintags.shape)
train_data.shape[0]*train_data.shape[1]

In [ ]:
bptt = params["seq_len"]

def totags(vec):
    tagvec = [nlp(TEXT.vocab.itos[x.detach().data]) for x in vec]
    tagvec = [y.tag_ for x in tagvec for y in x]
    tagvec = [tagstoi[x] for x in tagvec]
    assert len(tagvec) == len(vec)
    return tagvec

def get_tags_batch(source, i):
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    if model.training:
        target = trainingtags[i:i+seq_len].view(-1).long()
    else:
        target = validationtags[i:i+seq_len].view(-1).long()
    return data, target

In [ ]:
def batchify_tags(data, bsz):
    # Divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data

trainingtags = batchify_tags(traintags, 64)
validationtags = batchify_tags(valtags, 128)
print(trainingtags.shape, train_data.shape)

In [ ]:
ntokens = len(TEXT.vocab.stoi)

print(trainingtags.shape)
train_data.shape

In [ ]:
model = TransformerModel(d_model=params["embed_dim"], nhead=params["heads"], num_encoder_layers=params["layers"], ntokens=ntokens, output=ntags)
model = model.to("cuda")
model

In [ ]:
criterion = nn.CrossEntropyLoss()
lr = params["lr"]
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 2.0, gamma=0.65)

In [ ]:
import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_tags_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data.to(device), data.to(device), batch=batch, epoch=epoch)
        loss = criterion(output.view(-1, ntags), targets.to(device))
        acc = targets.to(device).eq(torch.argmax(output.view(-1, ntags), dim=1)).sum().detach().cpu().numpy()
        acc = acc / targets.shape[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 128
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.4f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | acc {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, acc))
            total_loss = 0
            start_time = time.time()

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    ntokens = len(TEXT.vocab.stoi)
    print("DATA SOURCE SIZE!", data_source.size(0))
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, bptt):
            data, targets = get_tags_batch(data_source, i)
            output = eval_model(data.to(device), data.to(device),batch=i, epoch=epoch)
            acc = targets.to(device).eq(torch.argmax(output.view(-1, ntags), dim=1)).sum().detach().cpu().numpy()
            acc = acc / targets.shape[0]
            output_flat = output.view(-1, ntags)
            total_loss += len(data) * criterion(output_flat, targets.to(device)).item()
    return total_loss / (len(data_source) - 1), acc

In [ ]:
best_val_loss = float("inf")
epochs = params["epochs"] # The number of epochs
best_model = None

print("STARTING:::::::.........")
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss, acc = evaluate(model, val_data)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid acc {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, acc))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

In [ ]:
test_loss, acc = evaluate(model, test_data)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test acc {:8.2f}'.format(
    test_loss, acc))
print('=' * 89)

In [ ]:
print(model.transformer.layers[0].attn.shape)
print(model.transformer.layers[0].attn_preactivation.shape)
print(model.transformer.layers[0].attn_activation.shape)

In [ ]:
frase = "the president of the united states is tall".split()


def generate_sent(frase, model, TEXT, words=5):
    """
    This function gets a list of a split string and 
    returns a new string with the next N number of words 
    predicted by the model
    """

    frasevec = TEXT.numericalize([frase]).to("cuda")
    model.eval()
    var = model(frasevec, frasevec)
    most = torch.argmax(var.squeeze(1), dim=1).cpu().detach().data.numpy()
    most = [itotags[x] for x in most]
    print(len(frase), len(most))
    result = [f"{frase[x]} : {most[x]}" for x in range(len(most))]
    return result


f = generate_sent(frase, model, TEXT)
for x in f:
    print(x)
len(f)

In [ ]:
frase = "the president of the united states is tall".split()

In [ ]:
frase = [nlp(x) for x in frase]
print(len(frase))
frase = [y.tag_ for x in frase for y in x]
frase

In [ ]:
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [ ]:
frase = "the president of the united states is tall"
a = tokenizer(frase)
print(a)
frase = [type(x) for x in a]
len(frase)

In [ ]:
itotags

In [ ]:
TEXT.numericalize([nlp.tokenizer("this is low, ok.")])

In [ ]:
TEXT.vocab.itos[12079]

In [ ]:
"." in TEXT.vocab.itos

In [ ]:
tokens = nlp.tokenizer("this is low, ok.")